In [ ]:
import time

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [ ]:
from sklearn.impute import SimpleImputer
# from sklearn.tree import RegressionTree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive/Colab/'):
        return '/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks' #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())

#Read Data

In [ ]:
test_df = pd.read_parquet('../data/1-original-data/test.parquet', engine = 'pyarrow')
print(test_df.shape)
len(test_df.columns)

(11363762, 190)


190

In [ ]:
test_df.head(5)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631315,0,0.010728,0.814497,0.0,0.168651,0.0,0.002347,...,-1,-1,-1,-1,0,NaN,NaN,-1,0.008281,-1
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.587042,0,0.011026,0.810848,0.0,0.241389,0.0,0.009132,...,-1,-1,-1,0,0,0.0,NaN,0,0.003753,0
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.609056,0,0.016390,1.004620,0.0,0.266976,0.0,0.004192,...,-1,-1,-1,0,0,0.0,NaN,0,0.002156,0
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-05-20,0.614911,0,0.021672,0.816549,0.0,0.188947,0.0,0.015325,...,-1,-1,-1,0,0,0.0,NaN,0,0.005206,0
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-06-15,0.591673,8,0.015923,0.810456,0.0,0.180035,0.0,0.011281,...,-1,-1,-1,0,0,0.0,NaN,0,0.007421,0


Drop columns with too many missing values/categorical variables that are not good predictors of defaulting

In [ ]:
test_df.drop(columns = ["B_31", "D_87", "D_88", "D_110", "B_39", "D_73", "B_42", "D_134", "B_29", "D_132", "D_76", "D_42", "D_142", "D_116", "D_63", "D_126", "D_66"], inplace = True)
len(test_df.columns)

173

Fill missing data (with RandomForest)

In [ ]:
test_df_randomForest = test_df.copy()

In [ ]:
tmp = test_df_randomForest.loc[:,['customer_ID','S_2']]
tmp.head(1)

,customer_ID,S_2
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19


In [ ]:
test_df_randomForest.drop(columns = ['customer_ID','S_2'], inplace = True)

In [ ]:
test_df_randomForest.columns

Index(['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_43',
       'D_44',
       ...
       'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_143',
       'D_144', 'D_145'],
      dtype='object', length=171)

In [ ]:
np.argsort(test_df_randomForest.isnull().sum(axis=0))

P_2      170
D_39      87
B_1       86
B_2      128
R_1       84
        ... 
D_140     38
D_141    127
D_143     53
D_144     21
D_145     28
Length: 171, dtype: int64

In [ ]:
sort_index = np.argsort(test_df_randomForest.isnull().sum(axis=0)).values
sort_index

array([170,  87,  86, 128,  84,  83,  82,  81,  80,  79,  78,  77, 129,
       125, 130,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63, 132,
        61,  73,  60,  89,  91, 117, 116, 115, 114, 113, 112, 111, 110,
       109, 108, 107, 106,  90, 105, 103, 102, 101, 100,  99,  98,  97,
        96,  95,  94,  93,  92, 104, 118,  59,  57, 151, 152,  24,  23,
        22, 153, 155,  19, 156,  17, 160,  15,  27, 161,  12, 162,  10,
         9, 163, 164, 165, 166,   4, 168,   2,   1,  13, 150,  29,  30,
        52,  51,  50, 135,  48, 136, 137,  45,  44,  55,  43,  56,  41,
        40, 141, 142, 144,  36,  35, 147,  33, 149,  31, 134,  54, 120,
       139, 131,  58,  85,  76,   7,   6,   3,  32,  11,  18, 123, 138,
        88,  47,  75, 124, 169, 122, 159,  25,  39,   0, 121, 119, 126,
       167, 158, 157, 154,  20, 145, 143, 148, 146,  62,  26, 133,  37,
        46,  16,  34,   5,  49,  14, 140,   8,  42,  74,  38, 127,  53,
        21,  28])

In [ ]:
for i in sort_index:
    t = time.time() # 记录时间

    # 构建新特征矩阵和新标签
    df = test_df_randomForest
    fillc = df.iloc[:,i] #即将被填充的特征，视为新目标变量
    # 找出能够用来训练的资料 以及 将要进行填补的资料
    ytrain = fillc[fillc.notnull()]
    ytest = fillc[fillc.isnull()]
    if len(ytest) == 0:
      continue
    df = pd.concat([df.iloc[:, :i], df.iloc[:, i+1:]], axis=1) #把新目标变量剔除，然后把原始标签进新特征矩阵
    
    # 在新特征矩阵中，对含有缺失值的列，用0进行填补。得到最终版特征矩阵
    df_0 = df.fillna(0)
    # print(ytrain.index, ytest.index)
    Xtrain = df_0.iloc[ytrain.index, :]
    Xtest = df_0.iloc[ytest.index, :]
    # 随机森林训练以及预测
    rfr = RandomForestRegressor(n_estimators=1, max_depth = 10, max_features = 80)
    rfr = rfr.fit(Xtrain, ytrain)
    ypredict = rfr.predict(Xtest)
    # 将填补结果放回数据集
    test_df_randomForest.iloc[test_df_randomForest.iloc[:,i].isnull(),i] = ypredict

    # 追踪记录
    dur = round((time.time() - t) / 60, 2)
    print(f"当前已完成第{i}个列的缺失值填补， 消耗时间 {dur} mins")

当前已完成第120个列的缺失值填补， 消耗时间 9.81 mins
当前已完成第139个列的缺失值填补， 消耗时间 10.44 mins
当前已完成第131个列的缺失值填补， 消耗时间 11.05 mins
当前已完成第58个列的缺失值填补， 消耗时间 11.06 mins
当前已完成第85个列的缺失值填补， 消耗时间 10.7 mins
当前已完成第76个列的缺失值填补， 消耗时间 11.09 mins
当前已完成第7个列的缺失值填补， 消耗时间 10.67 mins
当前已完成第6个列的缺失值填补， 消耗时间 10.75 mins
当前已完成第3个列的缺失值填补， 消耗时间 11.13 mins
当前已完成第32个列的缺失值填补， 消耗时间 6.74 mins
当前已完成第11个列的缺失值填补， 消耗时间 10.45 mins
当前已完成第18个列的缺失值填补， 消耗时间 11.14 mins
当前已完成第123个列的缺失值填补， 消耗时间 10.76 mins
当前已完成第138个列的缺失值填补， 消耗时间 11.03 mins
当前已完成第88个列的缺失值填补， 消耗时间 11.39 mins
当前已完成第47个列的缺失值填补， 消耗时间 10.52 mins
当前已完成第75个列的缺失值填补， 消耗时间 10.69 mins
当前已完成第124个列的缺失值填补， 消耗时间 11.07 mins
当前已完成第169个列的缺失值填补， 消耗时间 10.87 mins
当前已完成第122个列的缺失值填补， 消耗时间 10.64 mins
当前已完成第159个列的缺失值填补， 消耗时间 11.27 mins
当前已完成第25个列的缺失值填补， 消耗时间 10.62 mins
当前已完成第39个列的缺失值填补， 消耗时间 10.92 mins
当前已完成第0个列的缺失值填补， 消耗时间 10.82 mins
当前已完成第121个列的缺失值填补， 消耗时间 10.45 mins
当前已完成第119个列的缺失值填补， 消耗时间 10.63 mins
当前已完成第126个列的缺失值填补， 消耗时间 5.42 mins
当前已完成第167个列的缺失值填补， 消耗时间 3.24 mins
当前已完成第158个列的缺失值填补， 消耗时间 5.93 mins
当前已完成第157个

In [ ]:
test_df_randomForest[test_df_randomForest.isna().any(axis=1)]

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145


In [ ]:
test_df_randomForest = pd.concat([tmp, test_df_randomForest], axis = 1)

In [ ]:
test_df_randomForest.to_parquet(path = 'test_RandomForest(PreCompressed).parquet', engine = 'pyarrow')

Convert DataFrame into parquet format

In [ ]:
test_df_randomForest.to_parquet(path = '../data/4-PreCompressed/FilledWithRandomForest/test_RandomForest(PreCompressed).parquet', engine = 'pyarrow')

In [ ]:
test_df_randomForest.to_parquet(path = 'test_RandomForest(PreCompressed).parquet', engine = 'pyarrow')